<a href="https://colab.research.google.com/github/Chantifa/deterministcAutomation/blob/main/Table_export_RAMSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sys
import datetime
import time
from pathlib import Path
import json
import psycopg2
import warnings
warnings.filterwarnings('ignore')
import cx_Oracle

## SIP

In [ ]:
#Bitte Pfad anpassen für den Ornder bei dir also u-Nummer auswechseln!
path = "C:/users/u216983/SBB/TBET - Documents/06_VU-104_Prozessautomatisierung_J_K/Automatisierung & Predictive Maintenance im Störungsbereich/CSEM_Exports/"
path_to_json = path+"pw_sip.json"

with open(path_to_json, "r") as handler:
    info = json.load(handler)

username = info["username"]
password = info["password"]
service_name = info["service_name"]

#jdbc:oracle:thin:@(DESCRIPTION = (ADDRESS_LIST = (FAILOVER = ON) (LOAD_BALANCE = OFF)(ADDRESS = (PROTOCOL = TCP)(HOST = porasbbmgmscan.sbb.ch)(PORT = 1551)) 
#(ADDRESS = (PROTOCOL = TCP)(HOST = dorasbbmgmscan.sbb.ch oder porasbbmgmscan.sbb.ch)(PORT = 1551))(CONNECT_DATA = (SERVICE_NAME = DWH_PROD)))
con_string = "%s/%s@//SIP2-ODA01-ZK.sbb.ch/%s"%(username, password, service_name)
con = cx_Oracle.connect(con_string)

In [ ]:
sip_note = "SELECT * FROM FORM_SM_WF_NOTIZ_1"
df_sip_notiz = pd.read_sql(sip_note, con) # dataframe SIP2.0 Notiz_1

In [ ]:
df_sip_notiz.to_csv( 'SIP2_FORM_SM_WF_NOTIZ_1.csv', sep = ',',   encoding='utf-8')# write Datefram to csv file

In [ ]:
sip_doc = "SELECT * FROM FORM_SM_DMS_PROCESS_DOC_1"
df_sip_doc = pd.read_sql(sip_doc, con) # dataframe SIP2.0 doc_1

In [ ]:
df_sip_doc.to_csv( 'SIP2_FORM_SM_DMS_PROCESS_DOC_1.csv', sep = ',',   encoding='windows-1252')# write Datefram to csv file

In [ ]:
sip_worklog = "SELECT * FROM FORM_SM_WF_FS_WORKLOG_1"
df_sip_worklog = pd.read_sql(sip_worklog, con) # dataframe SIP2.0 worklogs

In [ ]:
df_sip_worklog.to_csv( 'SIP2_FORM_SM_WF_FS_WORKLOG_1.csv', sep = ',',   encoding='utf-8')# write Datefram to csv file

In [ ]:
sip_action_notiz = "SELECT * FROM FORM_SM_WF_ACTION_NOTIZ_1"
df_sip_order_notiz = pd.read_sql(sip_order_notiz, con) # dataframe SIP2.0 order note1

In [ ]:
df_sip_action_notiz.to_csv( 'SIP2_FORM_SM_WF_ACTION_NOTIZ_1.csv', sep = ',',   encoding='windows-1252')# write Datefram to csv file

In [ ]:
sip_order = "SELECT * FROM FORM_SM_WF_ORDER_1"
df_sip_order = pd.read_sql(sip_order, con) # dataframe SIP2.0 order note1

In [ ]:
df_sip_order.to_csv( 'SIP2_FORM_SM_WF_ORDER_1.csv', sep = ',',   encoding='windows-1252')# write Datefram to csv file

In [ ]:
sip_order_team = "SELECT * FROM FORM_SM_DMS_TEAM_HITLIST_1"
df_sip_order_team = pd.read_sql(sip_order_team, con) # dataframe SIP2.0 order team

In [ ]:
df_sip_order_team.to_csv( 'SIP2_FORM_SM_DMS_TEAM_HITLIST_1.csv', sep = ',', encoding='windows-1252')# write Datefram to csv file

In [ ]:
sip_order_operatingpoint = "SELECT * FROM FORM_SBB_DMS_OPERATINGPOINT_1"
df_sip_order_operatingpoint = pd.read_sql(sip_order_operatingpoint, con) # dataframe SIP2.0 order team

In [ ]:
df_sip_order_operatingpoint.to_csv( 'SIP2_FORM_SBB_DMS_OPERATINGPOINT_1.csv', sep = ',', encoding='windows-1252')# write Datefram to csv file

### RAMSI DB

In [ ]:
#Verbindung PostgreSql
path = "C:/users/u216983/SBB/TBET - Documents/06_VU-104_Prozessautomatisierung_J_K/Automatisierung & Predictive Maintenance im Störungsbereich/CSEM_Exports/"
path_to_json = path+"pw_postgresql.json"

with open(path_to_json, "r") as handler:
    info = json.load(handler)

    username = info["username"]
    password = info["password"]
    host = info["host"]
    database = info["db"]
    
#establishing the connection

con = psycopg2.connect(host=host, database=database ,user=username, password=password)
#Creating a cursor object using the cursor() method
cur = con.cursor()

#### Table RAMSI_LOADING.FAKT_SIP_TICKET

In [ ]:
sip_ticket = cur.execute("SELECT * FROM RAMSI_LOADING.FAKT_SIP_TICKET WHERE C_PROCESSID NOT LIKE 'D%'") # 
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
    
df_sip = pd.DataFrame(data=data, columns=cols) # dataframe SIP Ticket Tabelle

In [ ]:
df_sip.to_csv( 'RAMSI_LOADING_FAKT_SIP_TICKETS.csv', sep = ',', encoding='UTF-8')# write Datefram to csv file

#### Table RAMSI_LOADING.FAKT_SIP_WORKLOGS

In [ ]:
worklogs = cur.execute("SELECT * FROM RAMSI_LOADING.FAKT_SIP_WORKLOGS")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
    
df_fs_to_sip =pd.DataFrame(data=data, columns=cols) # dataframe foreign system to SIP

In [ ]:
df_fs_to_sip.to_csv( 'RAMSI_LOADING_FAKT_SIP_WORKLOGS.csv', sep = ',', encoding='utf-8')# write Datefram to csv file

#### Table RAMSI_LOADING.FAKT_SIP_EINGRIFFE

In [ ]:
eingriffe = cur.execute("SELECT * FROM RAMSI_LOADING.FAKT_SIP_EINGRIFFE")
d = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_sip_eingriffe = pd.DataFrame(d, columns=cols) # dataframe SIP Eingriffe

In [ ]:
df_sip_eingriffe.to_csv('RAMSI_LOADING_FAKT_SIP_EINGRIFFE.csv',sep = ',', encoding='utf-8')# write Datefram to csv file

#### Table RAMSI_LOADING.DIM_BETRIEBSPUNKT

In [ ]:
operational_location = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_BETRIEBSPUNKT")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_sip_bp = pd.DataFrame(data=data, columns=cols) # dataframe Operational Points Table DIM_BETRIEBSPUNKT (inventory tool)

In [ ]:
df_sip_bp.to_csv('RAMSI_LOADING_DIM_BP.csv',sep = ',', encoding='windows-1252')# write Datefram to csv file

#### Table RAMSI_LOADING.DIM_FOREIGN_SYSTEM

In [ ]:
foreign_system = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_FOREIGN_SYSTEM")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_fs = pd.DataFrame(data=data, columns=cols) # dataframe Table foreign system from dim (inventory tool)

In [ ]:
df_dim_fs.to_csv('RAMSI_LOADING_DIM_FOREIGN_SYSTEM.csv',sep = ',', encoding='windows-1252')# write Datefram to csv file

#### Table RAMSI_LOADING.DIM_SOLUTION

In [ ]:
dim_solution = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_SOLUTION")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_solution = pd.DataFrame(data=data, columns=cols) # dataframe Table dim_solution from dim (inventory tool)

In [ ]:
df_dim_solution.to_csv('RAMSI_LOADING_DIM_SOLUTION.csv',sep = ',', encoding='windows-1252')# write Datefram to csv file

#### Table RAMSI_LOADING.DIM_DIENSTSTELLE

In [ ]:
dim_dienststelle = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_DIENSTSTELLE")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_dienststelle = pd.DataFrame(data=data, columns=cols) # dataframe Table dim_dienststelle from dim (inventory tool)

In [ ]:
df_dim_dienststelle.to_csv('RAMSI_LOADING_DIM_DIENSTSTELLE.csv',sep = ',', encoding='windows-1252')# write Datefram to csv file

#### Table RAMSI_LOADING.DIM_MEASURE_1

In [ ]:
dim_measure_1 = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_MEASURE_1")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_measure_1 = pd.DataFrame(data=data, columns=cols) # dataframe Table dim_measure_1 from dim (inventory tool)

In [ ]:
df_dim_measure_1.to_csv('RAMSI_LOADING_DIM_MEASURE_1.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_MEASURE_2

In [ ]:
dim_measure_2 = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_MEASURE_2")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_measure_2 = pd.DataFrame(data=data, columns=cols) # dataframe Table dim_measure_2 from dim (inventory tool)

In [ ]:
df_dim_measure_2.to_csv('RAMSI_LOADING_DIM_MEASURE_2.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_MEASURE_3

In [ ]:
dim_measure_3 = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_MEASURE_3")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_measure_3 = pd.DataFrame(data=data, columns=cols) # dataframe Table dim_measure_2 from dim (inventory tool)

In [ ]:
df_dim_measure_3.to_csv('RAMSI_LOADING_DIM_MEASURE_3.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.FAKT_ESQ_EREIGNIS

In [ ]:
ereignis = cur.execute("SELECT * FROM RAMSI_LOADING.FAKT_ESQ_EREIGNIS")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_esq_ereignis = pd.DataFrame(data=data, columns=cols) # Datafram Table ESQ Ereignis 

In [ ]:
df_esq_ereignis.to_csv('RAMSI_LOADING_FAKT_ESQ_EREIGNIS.csv',sep = ',',encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_INVENTORYOBJECT

In [ ]:
inventory_object = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_INVENTORYOBJECT")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_inventoryobject = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM inventory object 

In [ ]:
df_dim_inventoryobject.to_csv('RAMSI_LOADING_DIM_INVENTORYOBJECT.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_INV

In [ ]:
inv = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_INV")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_inv = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM inventory

In [ ]:
df_dim_inv.to_csv('RAMSI_LOADING_DIM_INV.csv',sep = ',',encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_INV_AMDB_COMPONENT

In [ ]:
inv_amdb_component =  cur.execute("SELECT * FROM RAMSI_LOADING.DIM_INV_AMDB_COMPONENT")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_inv_amdb_component = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM component

In [ ]:
df_dim_inv_amdb_component.to_csv('RAMSI_LOADING_DIM_INV_AMDB_COMPONENT.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_REASON

In [ ]:
dim_reason = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_REASON")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_reason = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM inventory object 

In [ ]:
df_dim_reason.to_csv('RAMSI_LOADING_DIM_REASON.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_TECH_CONSEQ_1

In [ ]:
dim_tech_conseq_1 = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_TECH_CONSEQ_1")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_tech_conseq_1 = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM technical consequence

In [ ]:
df_dim_tech_conseq_1.to_csv('RAMSI_LOADING_DIM_TECH_CONSEQ_1.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_TECH_CONSEQ_2

In [ ]:
dim_tech_conseq_2 = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_TECH_CONSEQ_2")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_tech_conseq_2 = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM technical consequence

In [ ]:
df_dim_tech_conseq_2.to_csv('RAMSI_LOADING_DIM_TECH_CONSEQ_2.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_TECH_CONSEQ_3

In [ ]:
dim_tech_conseq_3 = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_TECH_CONSEQ_3")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_tech_conseq_3 = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM technical consequence

In [ ]:
df_dim_tech_conseq_3.to_csv('RAMSI_LOADING_DIM_TECH_CONSEQ_3.csv',sep = ',',encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_STELLWERK

In [ ]:
dim_stellwerk = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_STELLWERK")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_stellwerk = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM Stellwerke

In [ ]:
df_dim_stellwerk.to_csv('RAMSI_LOADING_DIM_STELLWERK.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_OPER_CONSEQ

In [ ]:
dim_oper_conseq = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_OPER_CONSEQ")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_oper_conseq = pd.DataFrame(data=data, columns=cols) # Datafram Table operational consequences

In [ ]:
df_dim_oper_conseq.to_csv('RAMSI_LOADING_DIM_OPER_CONSEQ.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_MILESTONE_PRIORITY

In [ ]:
dim_milestone_priority = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_MILESTONE_PRIORITY")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_milestone_priority = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM milstone priority

In [ ]:
df_dim_milestone_priority.to_csv('RAMSI_LOADING_DIM_MILESTONE_PRIORITY.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_INSIDENT_TYPE

In [ ]:
dim_incident_type = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_INCIDENT_TYPE")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_incident_type = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM incident type

In [ ]:
df_dim_incident_type.to_csv('RAMSI_LOADING_DIM_INCIDENT_TYPE.csv',sep = ',',encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_LAST_ASSIGNING_OU

In [ ]:
dim_assigning_ou = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_LAST_ASSIGNING_OU")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_assigning_ou = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM ASSIGNING

In [ ]:
df_dim_assigning_ou.to_csv('RAMSI_LOADING_DIM_LAST_ASSIGNING_OU.csv',sep = ',', encoding='windows-1252') # write Datefram to csv file

#### Table RAMSI_LOADING.DIM_ESI_CAUSE

In [ ]:
dim_esi_cause = cur.execute("SELECT * FROM RAMSI_LOADING.DIM_ESI_CAUSE")
data = cur.fetchall()
cols = []
for elt in cur.description:
    cols.append(elt[0])
df_dim_esi_cause = pd.DataFrame(data=data, columns=cols) # Datafram Table DIM ESI CAUSE

In [ ]:
df_dim_esi_cause.to_csv('RAMSI_LOADING_DIM_ESI_CAUSE.csv',sep = ',',encoding='windows-1252') # write Datefram to csv file